In [11]:
# from pyspatialml import Raster
# from sklearn.model_selection import train_test_split
# import pandas as pd 
# from sklearn.metrics import root_mean_squared_error
# from utils import (collect_tiff_paths, split_data_two)
# import pandas as pd
# from concurrent.futures import ProcessPoolExecutor


# def get_patch(df, idx):
#     return df.loc[idx:idx,:]

# def get_patch_paths(df, idx,cols=None):
#     if cols is None:
#         return df.loc[idx:idx,:].values[0].tolist()
#     else:
#         df = df[cols].copy()
#         return df.loc[idx:idx,:].values[0].tolist()


# def tif2df(patch_paths,cols):
#     ds = Raster(patch_paths)
#     ds.names = cols
#    # tb = ds.to_pandas()
#     return  ds.to_pandas()

# def process_path(i, valid_paths, FTCOLSC):
#     patch_paths = get_patch_paths(valid_paths, i, FTCOLSC)
#     di = tif2df(patch_paths, FTCOLSC)
#     di.dropna(subset=['ldtm'],inplace=True)
#     return di[FTCOLSC]


# def load_all_data(paths, FTCOLSC):
#     dilist = []
#     for i in range(len(paths)):
#         dilist.append(process_path(i,paths, FTCOLSC))
#     # Concatenate the dataframes
#     dfi = pd.concat(dilist, ignore_index=True)
#     return dfi 
# # make loadd_all_data parallel, and accout for the case that
# # # di.dropna( in  process_path may result in empty df as all rows are removed

In [3]:
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
from pyspatialml import Raster
from utils import collect_tiff_paths, split_data_two

def get_patch(df, idx):
  return df.loc[idx:idx, :]

def get_patch_paths(df, idx, cols=None):
  if cols is None:
      return df.loc[idx:idx, :].values[0].tolist()
  else:
      df = df[cols].copy()
      return df.loc[idx:idx, :].values[0].tolist()

def tif2df(patch_paths, cols):
  ds = Raster(patch_paths)
  ds.names = cols
  return ds.to_pandas()

def process_path(i, valid_paths, FTCOLSC, name='ldtm'):
  patch_paths = get_patch_paths(valid_paths, i, FTCOLSC)
  di = tif2df(patch_paths, FTCOLSC)
  di.dropna(subset=[name], inplace=True)
  if not di.empty:
      return di[FTCOLSC]
  return None

def load_all_data(paths, FTCOLSC,name):
  dilist = []
  with ProcessPoolExecutor() as executor:
      futures = [executor.submit(process_path, i, paths, FTCOLSC,name) for i in range(len(paths))]
      for future in futures:
          result = future.result()
          if result is not None and not result.empty:
              dilist.append(result)
  # Concatenate the dataframes
  if dilist:
      dfi = pd.concat(dilist, ignore_index=True)
  else:
      dfi = pd.DataFrame(columns=FTCOLSC)
  return dfi

# tonlesap_i = "N13E103"
# dpath = "/media/ljp238/12TBWolf/ARCHIEVE/ZOUT/datasets_TILES12_patches256/N13E103"
# dfpath = collect_tiff_paths(dpath)

produce data by without null values

In [ ]:
# tilename = 'N13E103'

# # tiles 1,2,3,4 drop bulls by pdem
# tilenames = os.listdir(tdpath)
# for tilename in tilenames:
#     dpath = f"{tdpath}/{tilename}"
#     dfpath = collect_tiff_paths(dpath)

#     if dfpath.shape == (1669, 20):
#         print(dfpath.shape)
#     else:
#         print(dfpath.shape)

#         fparquet = f'{dpath}/{tilename}_F_ldem_data.parquet'
#         if not isfile(fparquet):
#             FTCOLSC = dfpath.columns.tolist()
#             if 'S' in tilename:
#                 name = 'pdem'
#                 print(f'{tilename}@{name}')
#                 data = load_all_data(dfpath, FTCOLSC,name)
#             else:
#                 name = 'ldtm'
#                 print(f'{tilename}@{name}')
#                 data = load_all_data(dfpath, FTCOLSC,name)
#             data.to_parquet(fparquet)
#         else:
#             print('Already created!!!')